<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1b_S6_data_access_DS.ipynb">Accessing Sentinel-6 data via Data Store >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT <br>
**Authors:** Ben Loveday (EUMETSAT/Innoflair UG), Hayley Evers-King (EUMETSAT), Vinca Rosmorduc (CLS)

<div class="alert alert-block alert-success">
<h3>Learn Sentinel-6: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://eoportal.eumetsat.int/" target="_blank">A EUMETSAT Earth Observation Portal account</a>**
- SFTP access - you can need to request this via your EOP account. You will receive an email with your username and password.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1a Accessing <font color="#FF000000">operational</font> Sentinel-4 altimetry data via SFTP

### Data available

| Product Description  | Data Store collection ID| Product Navigator | 
|:--------------------:|:-----------------------:|:-------------:|
| **Sentinel-6 Poseidon-4 Altimetry level-1B Low Resolution (NTC)** | EO:EUM:DAT:0118 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0118?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| **Sentinel-6 Poseidon-4 Altimetry level-2 Low Resolution (NTC)** | EO:EUM:DAT:0120 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0120?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| **Sentinel-6 Poseidon-4 Altimetry level-2 Low Resolution (NRT)** | EO:EUM:DAT:0122 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0122?query=Altimetry%20NRT&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| Sentinel-6 Poseidon-4 Altimetry level-1A High Resolution | EO:EUM:DAT:0124 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0124?query=Altimetry&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| **Sentinel-6 Poseidon-4 Altimetry level-1B High Resolution (NTC)** | EO:EUM:DAT:0126 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0126?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| **Sentinel-6 Poseidon-4 Altimetry level-2 High Resolution (NTC)** | EO:EUM:DAT:0128 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0128?query=Altimetry%20NTC&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| **Sentinel-6 Poseidon-4 Altimetry level-2P High Resolution (NRT)** | EO:EUM:DAT:0134 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0134?query=Altimetry%20STC%202P&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| **Sentinel-6 Poseidon-4 Altimetry level-3 High Resolution (STC)** | EO:EUM:DAT:0139 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0139?query=Wave%20STC%202P&filter=satellite__Sentinel-6&results=20&s=advanced" target="_blank">link</a> |
| Sentinel-6 Climate Quality AMR-C level-2 | EO:EUM:DAT:0144 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:0144?query=&filter=satellite__Sentinel-6&results=24&s=advanced" target="_blank">link</a> |


### Learning outcomes

At the end of this notebook you will know;
* how to download data using the <font color="#138D75">**SFTP**</font> server

### Outline

Operational data from Sentinel-6 is currently available only via SFTP server. However, in the near future it will be available through the EUMETSAT Data Store, either via a web user interface (WebUI) or through code and command line interfaces with an Application Programming Interface (API). Reprocessed Sentinel-6 data is already available throug the Data Store (for more information see; **<a href="./1_1b_S6_data_access_DS.ipynb">Accessing Sentinel-6 data via Data Store</a>**). Here, for now, we will show you how to fetch data using SFTP.

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>

 1. [The products we need](#section1)
 1. [Downloading from the Sentinel-6 SFTP server](#section2)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
import configparser
import shutil
import os
import json
import tarfile
import glob
import fnmatch
import re
import paramiko #for downloading via SFTP

# Create a download directory for our Sentinel-6 products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## 1. <a id='section1'></a>The products we need for this module
[Back to top](#TOC_TOP)
    
</div>

We need a number of Sentinel-6-4 level-1, level-2, level-2P and level-3 products for the notebooks within the introductory part of the **learn-s6** module. We will download these from the SFTP server, fetching the latest versions of the products we need based on the orbit below;

In [2]:
selected_orbit = 213

<hr>
<div class="alert alert-info" role="alert">

## 2. <a id='section3'></a>Downloading from the Sentinel-6 SFTP server
[Back to top](#TOC_TOP)

</div>

To access Sentinel-6 data from the SFTP, we will use the `paramiko` library that we imported above. If you are working with the recommended Anaconda Python distribution and used the supplied environment file (environment.yml) to build this python environment (as detailed in the README), you will already have installed this. If not, you can install paramiko using;

`conda install -c conda-forge paramiko`

In order to allow us to download data from the SFTP server, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumetsat_sftp` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumetsat_sftp` in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"sftpURL": "files.eumetsat.int",
"sftpUser": "<your_username>",
"sftpPass": "<your_password>"
}
```

You must replace `<your_username>` and `<your_password>` with the information you received by email when you register for SFTP access (see the prerequisites box at the top of this page for more information.

*Note: your username and password are permanent, so you only need to do this once, but you should take care to never share them*

Make sure to save the file without any kind of extension.

Once you have done this, you can read in your credentials using the commands in the following cell.

#### Option 2: provide credentials directly

You can provide your credentials directly as follows;

`    ssh.connect(files.eumetsat.int, username=<your_username>, password=<your_password>)
`

*Note: this method is convenient in the short term, but is not really recommended as you have to put your username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.*

Let's load our credentials using **option 1**

In [3]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_sftp')) as json_file:
    credentials = json.load(json_file)

Now we have our connection credentials, we can download our Sentinel-6 files from the SFTP server. In this case we will get:

* the latest non-time critical Level-1 and Level-2 passes for our specified orbit
* one of the latest near real-time Level-2P pass for our specified orbit
* one of the latest short-time critical Level-3 aggregate products for the day corresponding to the Level-2P pass.

All of these are downloaded via the short (but a bit complicated!) routine below

In [4]:
with paramiko.SSHClient() as ssh:
    ssh.set_missing_host_key_policy( paramiko.AutoAddPolicy() )
    ssh.connect(credentials["sftpURL"], username=credentials["sftpUser"], password=credentials["sftpPass"])
    sftp = ssh.open_sftp()
    
    for directory, totfiles in zip(['/data/jcspdp/alt-ntc', '/data/jcspdp/alt-nrt', '/data/jcscnes/alt-l2p-l3'], [4, 2, 1]):
        nfiles_NTC = 0
        nfiles_NRT = 0
        nfiles_STC = 0
        acq_day = ''
        cycles = []
        sftp.chdir(directory)
        remote_files = sftp.listdir()
        
        for remote_file in remote_files:
            if '.tar' in remote_file and remote_file[75:78] == f'{selected_orbit}':
                cycles.append(int(remote_file[71:74]))
        
        for remote_file in remote_files:
            if fnmatch.fnmatch(remote_file, f'*P4*_{str(max(cycles)).zfill(3)}_{selected_orbit}_*NT*.tar') and "1A" not in remote_file and nfiles_NTC < totfiles:
                print(f'Fetching: {remote_file}')
                sftp.get(remote_file, os.path.join(download_dir, remote_file))
                nfiles_NTC = nfiles_NTC + 1
            elif (fnmatch.fnmatch(remote_file, f'*P4*_LR_*_{selected_orbit}_*NR*.tar')
              or fnmatch.fnmatch(remote_file, f'*P4_2P*{selected_orbit}_*NR*.tar')
              or fnmatch.fnmatch(remote_file, f'*P4*_HR_*_{selected_orbit}_*NR*.tar')) and nfiles_NRT < totfiles:
                print(f'Fetching: {remote_file}')
                sftp.get(remote_file, os.path.join(download_dir, remote_file))
                nfiles_NRT = nfiles_NRT + 1
                acq_day = f"{remote_file[18:27]}000000_{remote_file[18:27]}235959"
            elif fnmatch.fnmatch(remote_file, f'*P4*{acq_day}*_________*ST*.tar') and nfiles_STC < totfiles:
                print(f'Fetching: {remote_file}')
                sftp.get(remote_file, os.path.join(download_dir, remote_file))
                nfiles_STC = nfiles_STC + 1


Fetching: S6A_P4_1B_HR______20230625T222845_20230625T232458_20230718T171046_3373_096_213_106_EUM__OPE_NT_F08.SEN6.tar
Fetching: S6A_P4_2__LR______20230625T222845_20230625T232458_20230717T025141_3373_096_213_106_EUM__OPE_NT_F08.SEN6.tar
Fetching: S6A_P4_1B_LR______20230625T222845_20230625T232458_20230716T063254_3373_096_213_106_EUM__OPE_NT_F08.SEN6.tar
Fetching: S6A_P4_2__HR______20230625T222845_20230625T232458_20230718T171054_3373_096_213_106_EUM__OPE_NT_F08.SEN6.tar
Fetching: S6A_P4_2__HR______20230715T191843_20230715T192843_20230715T214322_0600_098_213_107_EUM__OPE_NR_F08.SEN6.tar
Fetching: S6A_P4_2__LR______20230715T190846_20230715T210410_20230715T212134_6924_098_213_107_EUM__OPE_NR_F08.SEN6.tar
Fetching: S6A_P4_3__HR______20230709T000000_20230709T235959_20230716T111510__________________CNES_OPE_ST____.SEN6.tar
Fetching: S6A_P4_2P_HR______20230715T182549_20230715T183046_20230715T202503_0297_098_213_107_CNES_OPE_NR_F08.SEN6.tar


The products are downloaded as zipped files, so lets unzip them and remove the zips.

In [5]:
for fdst in glob.glob(os.path.join(download_dir, "*.tar")):
    tar = tarfile.open(fdst, "r:")
    tar.extractall(download_dir)
    tar.close()
    os.remove(fdst)

Now we are ready to go. If you check the `./products` directory, you should see all the required SAFE format, ".SEN6" directories. Now you can continue to work through the rest of the notebooks in this repository and learn more about working with Sentinel-6 data.

<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1b_S6_data_access_DS.ipynb">Accessing Sentinel-6 data via Data Store >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>